In [13]:
import os
import librosa
import pandas as pd
import numpy as np
import csv
import keras
import tkinter as tk
from tkinter import *
from mutagen.mp3 import MP3
from pygame import mixer
from pydub import AudioSegment

import warnings
warnings.filterwarnings('ignore')
from keras.models import load_model
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [14]:
def detectEmotion(path):
    songname = f'{path}'
    print (songname)
    base=os.path.basename(path)
    print (base)
    name=os.path.splitext(base)[0]
    print (name)
    header = 'chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
    for i in range(1, 21):
         header += f' mfcc{i}'

    header = header.split()

    file = open('song_data.csv', 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)
        
    #feature extraction using Librosa library
    y, sr = librosa.load(songname, mono=True, duration=30)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}' 
    print(to_append)
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    file = open('song_data.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())

    model = load_model('my_model.h5')
    data_from_csv = pd.read_csv("song_data.csv")
    data = data_from_csv
    print (data)
    data.shape
    scaler = joblib.load("scaler.save") 
    X = scaler.transform(np.array(data.iloc[:,:], dtype = float))
    print(X)
    ynew = model.predict(np.array(X))
    
    #ynew = model.predict(np.array(data))
    print (ynew)
    if np.argmax(ynew)==0:
        print ("Calm")
        return "calm"
    elif np.argmax(ynew)==1:
        print ("Happy")
        return "happy"
    elif np.argmax(ynew)==2:
        print ("Sad")
        return "sad"


detectEmotion("E:/mp3/Lewis Capaldi - Someone You Loved (Mp3 + Lyrics).mp3")

E:/mp3/Lewis Capaldi - Someone You Loved (Mp3 + Lyrics).mp3
Lewis Capaldi - Someone You Loved (Mp3 + Lyrics).mp3
Lewis Capaldi - Someone You Loved (Mp3 + Lyrics)
0.22539016604423523 1121.4820891095994 1362.8318388046512 1996.229595739406 0.054935419891640865
   chroma_stft  spectral_centroid  spectral_bandwidth      rolloff  \
0      0.22539        1121.482089         1362.831839  1996.229596   

   zero_crossing_rate       mfcc1       mfcc2      mfcc3      mfcc4    mfcc5  \
0            0.054935 -253.589691  149.077972  30.465044  14.435976  4.00155   

   ...    mfcc11    mfcc12    mfcc13    mfcc14    mfcc15   mfcc16    mfcc17  \
0  ... -5.661661 -7.093256 -2.426906 -0.905724 -0.951321 -1.18342 -6.258499   

     mfcc18    mfcc19    mfcc20  
0 -3.861773 -3.314568 -3.495067  

[1 rows x 25 columns]
[[-1.19110951 -1.02893342 -1.27434017 -1.12899972 -0.62955515 -0.14135335
   1.17988239  1.10396266 -0.32235691 -0.10556834 -0.06651275 -1.03618158
   0.87950759 -0.33326606 -0.8985735   0.

'sad'